In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.cross_validation import train_test_split
import string

/home/suraj/.virtualenvs/tf/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_path = '../data/titanic/'

In [3]:
train = pd.read_csv( os.path.join(data_path, 'train.csv') )
test = pd.read_csv( os.path.join(data_path, 'test.csv') )
sub = pd.read_csv( os.path.join(data_path, 'gender_submission.csv') )

In [4]:
test['Fare'][152] = train['Fare'].mean()

/home/suraj/.virtualenvs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
train = train.drop('Ticket', axis=1)
train = train.drop('Embarked', axis=1)
train = train.drop('PassengerId', axis=1)
train = train.drop('Cabin', axis=1)
train['FamilySize'] = train['SibSp'] + train['Parch']
train['Total Fare'] = (train['FamilySize']+1) * train['Fare']
train = train.drop('Fare', axis=1)


test_pID = test['PassengerId']
test = test.drop('Ticket', axis=1)
test = test.drop('Embarked', axis=1)
test = test.drop('PassengerId', axis=1)
test = test.drop('Cabin', axis=1)
test['FamilySize'] = test['SibSp'] + test['Parch']
test['Total Fare'] = (test['FamilySize']+1) * test['Fare']
test = test.drop('Fare', axis=1)


In [7]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,FamilySize,Total Fare
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,1,14.5000
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,1,142.5666
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,0,7.9250
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,1,106.2000
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,0,8.0500


In [8]:
title_list = ['Mr', 'Mrs', 'Dr', 'Miss', 'Master', 'Rev', 'Ms', 'Capt', 'Sir', 'Major',\
              'Don', 'Mlle', 'Mme', 'Jonkheer', 'Countess', 'Col']

In [9]:
# Reference : https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/

def title_in_name(name, target_list):
    for title in title_list:
        if name.find(title) != -1:
            return title
    print(name)
    return np.nan


In [10]:
train['Title'] = train['Name'].map(lambda x: title_in_name(x, title_list))
train = train.drop('Name', axis=1)

test['Title'] = test['Name'].map(lambda x: title_in_name(x, title_list))
test = test.drop('Name', axis=1)


In [11]:
#train.head()
test.head()

,Pclass,Sex,Age,SibSp,Parch,FamilySize,Total Fare,Title
0,3,male,34.5,0,0,0,7.8292,Mr
1,3,female,47.0,1,0,1,14.0000,Mr
2,2,male,62.0,0,0,0,9.6875,Mr
3,3,male,27.0,0,0,0,8.6625,Mr
4,3,female,22.0,1,1,2,36.8625,Mr


In [12]:
def replace_titles(x):
    title = x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ['Countess', 'Mme', 'Mme.']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms', 'Mlle.', 'Ms.']:
        return 'Miss'
    elif title == 'Dr':
        if x['Sex'] == 'male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title
    
train['Title'] = train.apply(replace_titles, axis=1)
test['Title'] = test.apply(replace_titles, axis=1)

In [13]:
train['Sex'] = (train['Sex'] == 'male').astype('int')
test['Sex'] = (test['Sex'] == 'male').astype('int')

In [14]:
X = train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'FamilySize', 'Title', 'Total Fare']]
Y = train[['Survived']]

X_test = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'FamilySize', 'Title', 'Total Fare']]


In [15]:
test_split = 0.1
X_train, X_CV, Y_train, Y_CV = train_test_split(X, Y, test_size=test_split, random_state=42)


print('Train : ')
print(X_train.shape)
print(Y_train.shape)
print('CV : ')
print(X_CV.shape)
print(Y_CV.shape)

Train : 
(801, 8)
(801, 1)
CV : 
(90, 8)
(90, 1)


In [16]:
def get_means_age(df):
    sum_val={'Mr':0, 'Mrs':0, 'Miss':0, 'Master':0}
    count_val={'Mr':0, 'Mrs':0, 'Miss':0, 'Master':0}
    mean_val={'Mr':0, 'Mrs':0, 'Miss':0, 'Master':0}
    
    for _, row in df.iterrows():
        if not np.isnan(row['Age']):
            sum_val[row['Title']] += row['Age']
            count_val[row['Title']] += 1
    
    for key in sum_val.keys():
        mean_val[key] = sum_val[key] / count_val[key]
        
    return mean_val

mean_ages = get_means_age(train)

def replace_nan_ages(x):
    if np.isnan(x['Age']):
        x['Age'] = mean_ages[x['Title']]
    return x




In [17]:
X_train = X_train.apply(replace_nan_ages, axis=1)
X_CV = X_CV.apply(replace_nan_ages, axis=1)
X_test = X_test.apply(replace_nan_ages, axis=1)


In [18]:
title = {'Mr':0, 'Master':1, 'Miss':2, 'Mrs':3}

def title2num(x):
    return title[ x['Title'] ]

X_train['Title'] = X_train.apply(title2num, axis=1)
X_CV['Title'] = X_CV.apply(title2num, axis=1)
X_test['Title'] = X_test.apply(title2num, axis=1)

In [19]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,FamilySize,Title,Total Fare
165,3,1,9.000000,0,2,2,1,61.5750
541,3,0,9.000000,4,2,6,2,218.9250
625,1,1,61.000000,0,0,0,0,32.3208
388,3,1,33.547259,0,0,0,0,7.7292
76,3,1,33.547259,0,0,0,0,7.8958


In [20]:
Y_train.head()

,Survived
165,1
541,0
625,0
388,0
76,0


In [21]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_CV = np.array(X_CV)
Y_CV = np.array(Y_CV)

X_test = np.array(X_test)

Y_train =  Y_train.ravel()
Y_CV =  Y_CV.ravel()

# Model Training

In [22]:
X_train = np.concatenate((X_train, X_CV))
Y_train = np.concatenate((Y_train, Y_CV))


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn import metrics
import matplotlib.pyplot as plt
from scipy.stats import randint

/home/suraj/.virtualenvs/tf/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [24]:
params = {"n_estimators": [1000, 5000],
          "criterion": ["gini", "entropy"],
          "min_samples_leaf":randint(1, 9),
          "max_depth": [1, None]}

In [25]:
clf = RandomForestClassifier(verbose=1, n_jobs=8)

In [26]:
best_model = RandomizedSearchCV(clf, params, cv=5)

In [27]:
best_model.fit(X_train, Y_train)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    2.2s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    3.1s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    4.1s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    5.3s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    6.4s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    6.4s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.4s
[Paral

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    2.1s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    2.9s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    4.0s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    5.0s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    6.2s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    6.2s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.4s
[Paral

[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    2.5s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    2.5s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    2.2s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    3.4s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    4.5s
[Par

[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    1.8s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    2.2s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    2.2s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    1.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0

[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    1.5s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    1.4s
[Pa

...

[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    1.9s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.8s
[Para

[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    1.3s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0.4s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0.5s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapse

[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    2.5s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    2.5s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    2.7s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    3.7s
[Par

[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    1.7s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    2.2s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    2.2s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    2.8s
[Par

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=8,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'max_depth': [1, None], 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0907898b70>, 'criterion': ['gini', 'entropy'], 'n_estimators': [1000, 5000]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [28]:
best_model.best_score_

0.8249158249158249

In [29]:
best_model.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'min_samples_leaf': 2,
 'n_estimators': 5000}

In [34]:
clf = RandomForestClassifier(verbose=1, n_jobs=8, **best_model.best_params_)

In [36]:
clf.fit(X_train, Y_train)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    1.7s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    2.4s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    3.3s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    4.3s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    6.0s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    7.5s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    7.5s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5000, n_jobs=8,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [37]:
clf.feature_importances_

array([0.10262099, 0.2288377 , 0.20952098, 0.03591142, 0.02160481,
       0.05655003, 0.08467158, 0.2602825 ])

In [ ]:
'''
print('Accuracy : ', clf.score(X_CV, Y_CV))

fpr, tpr, thresholds = metrics.roc_curve(Y_CV, Y_pred_CV, pos_label=1)
roc_auc = metrics.auc(fpr, tpr)
print('AUC       : ', roc_auc)

f1 = metrics.f1_score(Y_CV, Y_pred_CV, average='binary')
print('F1 score  : ', f1)

cm = metrics.confusion_matrix(Y_CV, Y_pred_CV)
print('Confusion Matrix : ')
print(cm)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.grid()
'''

# Making Prediction

In [41]:
prediction = clf.predict(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    1.8s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    1.8s finished


In [42]:
id_list = np.array(test_pID)

In [43]:
submission = {'PassengerId': id_list,
              'Survived': prediction}

In [44]:
df = pd.DataFrame.from_dict(submission)

In [47]:
df.to_csv('submission.csv', index=False)